In [2]:
%matplotlib qt4
from __future__ import division
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from models import tools

sns.set_style("ticks", {"legend.frameon": True})
mpl.rcParams['axes.color_cycle'] = ['#02A5F4', 'orange', 'green']

In [10]:
def func_learn(x, r=4, s=0):
    return 1 - (1 - s)*np.exp(-r*x)
def func_forget(x, r=4, s=1):
    return s * np.exp(-r * x)

### Styles

In [3]:
line_dashed = {'color': '#000000', 'linewidth': '0.8', 'linestyle': 'dashed'}
text_label = {'color': '#000000', 'fontsize': 11, 'fontstyle': 'italic'}

### Learning and Forgetting Curve

In [21]:
points = np.arange(0, 1, 0.001)
frame, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(7, 3), dpi=160)

ax1.plot(points, func_learn(points))
ax2.plot(points, func_forget(points))

ax1.set_xlabel('Time')
ax1.set_ylabel('Memory strength')
ax2.set_xlabel('Time')

ax1.plot([0, 1], [1, 1], **line_dashed)
ax2.plot([0, 1], [1, 1], **line_dashed)

ax1.annotate('$K_{max}$', xy=(1, 0.5), xytext=(0.8, 1.07), **text_label)
ax2.annotate('$K_{max}$', xy=(1, 0.5), xytext=(0.8, 1.07), **text_label)

ax1.get_xaxis().set_ticks([])
ax2.get_xaxis().set_ticks([])

plt.ylim([0, 1.15])

sns.despine()
plt.tight_layout()

### Spacing Effect

In [128]:
def r(t, repeat=1, step=200, rang=100000, spacing_rate=0.2, decay_rate=0.18):
    times = t.copy()
    vals = []
    results = []
    last = 0
    for _ in range(repeat):
        for i in range(0, rang // repeat, step):
            times += step
            param = np.concatenate([times, [1]])
            strength = tools.memory_strength(param, spacing_rate, decay_rate)
            retrieval_prob = strength # tools.retrieval_prob(strength, tau=-1, s=0.3)
            results.append(retrieval_prob)
            vals.append(last+i)
        last = vals[-1]
        times = np.concatenate([times, t.copy()])
    return vals, results

In [88]:
t1 = np.array(list(reversed(range(50))))
t2 = np.array(list(reversed(range(150))))

x1, y1 = r(t1, repeat=3, spacing_rate=0.1, decay_rate=0.2)
x2, y2 = r(t2, repeat=1, spacing_rate=0.1, decay_rate=0.2)

In [104]:
fig = plt.figure(num=None, figsize=(6, 3.5), dpi=160)
ax = plt.subplot(111)

ax.plot(x1, y1, label='student $s_1$')
ax.plot(x2, y2, label='student $s_2$')
ax.get_xaxis().set_ticks([])

plt.xlabel('Time')
plt.ylabel('Memory strength')

# Put a legend to the right of the current axis
legend = ax.legend(loc='lower right', bbox_to_anchor=(1, 0.1), prop={'size': 12})
legend.get_frame().set_linewidth(1)

plt.xticks([0, 33333, 66666], ['$t_0$', '$t_1$', '$t_2$'], **text_label)

sns.despine()
plt.xlim([0, max(x1)])
plt.ylim([0, 1])
plt.tight_layout()

### Knowledge Levels

In [22]:
points = np.arange(0, 1.33, 0.001)

def func(x, r=5):
    if x > 0.5:
        return func_learn(0.5, r=r) * np.exp(-(r-1) * (x - 0.5))
    return func_learn(x, r=r)

plt.figure(num=None, figsize=(6, 3), dpi=160)
frame, = plt.plot(points, [func(x) for x in points])

plt.plot([0, 1.33], [0.8, 0.8], **line_dashed)
plt.plot([0, 1.33], [0.6, 0.6], **line_dashed)
plt.plot([0, 1.33], [0.4, 0.4], **line_dashed)
plt.plot([0, 1.33], [0.2, 0.2], **line_dashed)

plt.annotate('Atomaticity', xy=(1, 0.8), xytext=(1, 0.85), **text_label)
plt.annotate('Recall', xy=(1, 0.6), xytext=(1, 0.65), **text_label)
plt.annotate('Recognition', xy=(1, 0.4), xytext=(1, 0.45), **text_label)
plt.annotate('Femiliarity', xy=(1, 0.2), xytext=(1, 0.25), **text_label)

plt.xlabel('Time')
plt.ylabel('Memory strength')
frame.axes.get_xaxis().set_ticks([])
sns.despine()
plt.tight_layout()

### Time Effect Functions

In [161]:
points = np.arange(10, 1e6, 5)

f_log = lambda t: 1.8 - 0.123 * np.log(t)
f_exp = lambda t: 1.58 * np.exp(-10e-3 * np.sqrt(t))
f_poly = lambda t: 3 / t**0.3

fig = plt.figure(num=None, figsize=(6, 3.5), dpi=160)
ax = plt.subplot(111)

ax.plot(points, f_log(points), label='$1.8 - 0.12 \cdot \log(t)$')
ax.plot(points, f_exp(points), label='$1.6 \cdot e^{-0.01 \sqrt{t}}$')
ax.plot(points, f_poly(points), label='$3 / t^{0.3}$')

plt.xscale('log')

# Shrink current axis by 20%
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
legend = ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), prop={'size': 12})
legend.get_frame().set_linewidth(1)

plt.tight_layout()
plt.subplots_adjust(right=0.62)

### Gradient Descent

In [157]:
from mpl_toolkits.mplot3d import axes3d

fig = plt.figure(figsize=(6, 4), dpi=160)
ax = fig.gca(projection='3d')

X, Y, Z = axes3d.get_test_data(0.05)
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.RdYlGn,
                       linewidth=0, antialiased=True)
ax.set_zlim(-130, 130)
ax.set_xlim(-29, 29)
ax.set_ylim(-29, 29)
ax.set_xlabel('p1')
ax.set_ylabel('p2')
ax.set_zlabel('J(p)')

ax.plot([17, 13, 14, 9, 10, 6, 5],
        [10, 8, 3, 3, -1, -3, -6], '.-',
        zs=[70, 60, 40, 15, -10, -30, -45],
        zdir='z', color='black', linewidth=1)

plt.show()
plt.tight_layout()

### Sigmoid Function

In [58]:
points = np.arange(-6, 6, 0.1)

fig = plt.figure(num=None, figsize=(6, 3.5), dpi=120)
ax = plt.subplot(111)

ax.plot(points, tools.sigmoid(points), label='$c = 0$')
ax.plot(points, 0.3 + 0.7*tools.sigmoid(points), label='$c = 0.3$')

plt.xlabel('$m$')
plt.ylabel('$\sigma_c(m)$', rotation='horizontal')

ax.xaxis.set_label_coords(0.98, 0.1)
ax.yaxis.set_label_coords(0.56, 0.95)

plt.xlim([-6, 6])

legend = ax.legend(loc='upper left', prop={'size': 12})
legend.get_frame().set_linewidth(1)

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
ax.spines['left'].set_position('center')
#ax.spines['bottom'].set_position('center')

# Eliminate upper and right axes
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Show ticks in the left and lower axes only
ax.axes.get_yaxis().set_ticks([0.2, 0.4, 0.6, 0.8, 1])
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

plt.tight_layout()
plt.show()